# Train model on Food 101 DataFrame

## Import Food101 Data And Some Pre_Preocessing

In [2]:
# import some neesassary libraries
import tensorflow as tf
from tensorflow . keras . applications import MobileNetV2
from tensorflow . keras . preprocessing . image import ImageDataGenerator
from tensorflow . keras . layers import Dense, GlobalAveragePooling2D
from tensorflow . keras . models import Model
from tensorflow . keras . optimizers import Adam

In [3]:
# import food101 dataframe
import tensorflow_datasets as tfds
# load food101 data
(train_ds, val_ds), ds_info = tfds.load('food101' , split=['train', 'validation'] , shuffle_files = True , as_supervised = True, with_info = True)

In [4]:
# image size we want is 224*224 . also batch_size or pic we give to model is in group of 32 pictures
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
# do pre_process on image
def preprocess(image, label):
    image = tf . image . resize(image, IMG_SIZE)
    image = tf . keras . applications . mobilenet_v2.preprocess_input(image)
    return image, label
# make dataset of train and validation
train_ds = train_ds . map(preprocess) . batch(BATCH_SIZE) . prefetch(buffer_size=tf . data . AUTOTUNE)
val_ds = val_ds . map(preprocess) . batch(BATCH_SIZE) . prefetch(buffer_size=tf . data . AUTOTUNE)

## Train Model Using Transfer_learninng

### Part1 - Train Model Without Any Trainable part

In [5]:
base_model = MobileNetV2(input_shape = (224, 224, 3) , include_top = False , weights = 'imagenet')
base_model . trainable = False

9406464/9406464 [==============================] - 38s 4us/step


In [6]:
# make model here
# we use 1 hidden layer with 1024 norouns
inputs = tf . keras . Input(shape = (224, 224, 3))
x = base_model(inputs, training = False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
outputs = Dense(ds_info . features['label'] . num_classes, activation = 'softmax')(x)
model = Model(inputs, outputs)

In [7]:
# make and compile model
model . compile(optimizer = Adam() , loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [9]:
# train model without any trainable part of MobileNetV2
history = model . fit(train_ds , epochs = 5, validation_data = val_ds)

Epoch 1/5
2368/2368 [==============================] - 1142s 481ms/step - loss: 2.0394 - accuracy: 0.4856 - val_loss: 1.5759 - val_accuracy: 0.5809
Epoch 2/5
2368/2368 [==============================] - 1119s 473ms/step - loss: 1.5641 - accuracy: 0.5899 - val_loss: 1.5074 - val_accuracy: 0.5988
Epoch 3/5
2368/2368 [==============================] - 1097s 463ms/step - loss: 1.3147 - accuracy: 0.6459 - val_loss: 1.5514 - val_accuracy: 0.5996
Epoch 4/5
2368/2368 [==============================] - 1091s 461ms/step - loss: 1.0893 - accuracy: 0.6997 - val_loss: 1.6367 - val_accuracy: 0.5937
Epoch 5/5
2368/2368 [==============================] - 1092s 461ms/step - loss: 0.8812 - accuracy: 0.7505 - val_loss: 1.7518 - val_accuracy: 0.5856


### Part2 - All Layers Are Traineable But 100 First Layer Is Freeze

In [10]:
base_model . trainable = True
for layer in base_model . layers[:100]:
    layer . trainable = False
# compile model
model.compile(optimizer = Adam(1e-5) , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])
# make and fit model for second part and get the final result
history_fine = model.fit(train_ds , epochs = 5 , validation_data = val_ds)

Epoch 1/5
2368/2368 [==============================] - 1573s 662ms/step - loss: 0.5901 - accuracy: 0.8305 - val_loss: 1.5871 - val_accuracy: 0.6320
Epoch 2/5
2368/2368 [==============================] - 1652s 698ms/step - loss: 0.4270 - accuracy: 0.8819 - val_loss: 1.6092 - val_accuracy: 0.6387
Epoch 3/5
2368/2368 [==============================] - 1647s 696ms/step - loss: 0.3309 - accuracy: 0.9134 - val_loss: 1.6276 - val_accuracy: 0.6413
Epoch 4/5
2368/2368 [==============================] - 1666s 704ms/step - loss: 0.2596 - accuracy: 0.9372 - val_loss: 1.6622 - val_accuracy: 0.6443
Epoch 5/5
2368/2368 [==============================] - 1596s 674ms/step - loss: 0.2024 - accuracy: 0.9547 - val_loss: 1.7205 - val_accuracy: 0.6461


In [11]:
# save model
model . save('finalmodel.h5')

## Make The DataFrame That Have 3 Columns : Photo_Name , Photo , Label , Predicted

### Part0 - Pre_Process Images

In [12]:
# function that use predict and predict class of picture
def make_prediction(photo_data):
    prediction = model . predict(photo_data)
    return prediction

In [13]:
# import some library for pre_process on data
import os
import pandas as pd
import tensorflow as tf
from PIL import Image
import io
import numpy as np
# Function to preprocess the image
def preprocess_image(image_bytes):
    # Convert byte string to bytes
    # open image
    image = Image . open(io . BytesIO(eval(image_bytes)))
    # resize image to 224*224 to all image have same size
    image = image . resize((224, 224))
    # convert image into numpy array to give this array into model to predict
    image_array = np . array(image)
    # create a batch as we give 224*224*3 array into model
    image_array = np . expand_dims(image_array, axis=0)
    # normalize the input pic
    image_array = image_array / 255.0
    return image_array

### Part1 - Make The Predictions And Give This List to DataFrame

In [14]:
# Path to destination folder
folder_path = 'excel'
df = pd . read_csv('final1.csv')
# Make predictions
predictions = []
for img_raw in df['Photo']:
    try:
        img_array = preprocess_image(img_raw)
        prediction = make_prediction(img_array)
        predicted_class = tf .argmax(prediction, axis = -1) . numpy()[0]
        predictions.append(predicted_class)
    except :
        # we may have some pics that have None value and so our for loop got exception
        # so we add this part to prevent crush during prediction
        predictions . append(None)
# Add predictions to the DataFrame
df['predict'] = predictions

1/1 [==============================] - 0s 472ms/step


In [22]:
# after make final dataframe, save it . also we drop column of Photo to reduce size of save csv file
df2 = df . copy()
df2 . drop(columns = ['Photo'] , inplace = True)
df2 . to_csv('final3.csv' , index = False)